In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
tf.random.set_seed(1234)
np.random.seed(1234)

## Loading MNIST Data

In [ ]:
# Loading Data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
# Adding Channel Lenght: Expanding from (28x28) to (28x28x1)
x_train = tf.expand_dims(x_train, -1)
x_test = tf.expand_dims(x_test, -1)
# Creating validation Subset
x_valid = x_train[50000:]  
y_valid = y_train[50000:]  

x_train = x_train[:50000]
y_train = y_train[:50000]


## Creating Dataset Objects

In [ ]:
#Training
# ------------------------------
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# Shuffle
train_dataset = train_dataset.shuffle(buffer_size=x_train.shape[0])

# Normalize images
def normalize_img(x_, y_):
    return tf.cast(x_, tf.float32) / 255., y_

# 1-hot encoding <- for categorical cross entropy
def to_categorical(x_, y_):
    return x_, tf.one_hot(y_, depth=10)

train_dataset = train_dataset.map(to_categorical)
train_dataset = train_dataset.map(normalize_img)

# Divide in batches
bs = 32
train_dataset = train_dataset.batch(bs)
# Repeat
# Without calling the repeat function the dataset 
# will be empty after consuming all the images
train_dataset = train_dataset.repeat()

In [ ]:
#Validation   
# -----------------------
valid_dataset = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
# Normalize images
valid_dataset = valid_dataset.map(normalize_img)
# Enconding
valid_dataset = valid_dataset.map(to_categorical)
# Divide in batches
valid_dataset = valid_dataset.batch(bs)
# Repeat
valid_dataset = valid_dataset.repeat()

In [ ]:
#Testing 
# -------------------
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.map(normalize_img)
test_dataset = test_dataset.map(to_categorical)
test_dataset = test_dataset.batch(1)

## Creating the model

In [ ]:
# Implementation of the original Lenet5 
lenet = tf.keras.Sequential()
lenet.add(tf.keras.layers.Conv2D(6, kernel_size=5, strides=1,  activation='tanh', input_shape=(28,28,1), padding='same'))
lenet.add(tf.keras.layers.AveragePooling2D())
lenet.add(tf.keras.layers.Conv2D(16, kernel_size=5, strides=1, activation='tanh', padding='valid'))
lenet.add(tf.keras.layers.AveragePooling2D())
lenet.add(tf.keras.layers.Flatten())
lenet.add(tf.keras.layers.Dense(units=120, activation='tanh'))
lenet.add(tf.keras.layers.Dense(units=84, activation='tanh'))
lenet.add(tf.keras.layers.Dense(units=10, activation = 'softmax'))

## Training Options

In [ ]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
lenet.compile(optimizer=optimizer, loss=loss, metrics=metrics)

## Checkpoints

In [ ]:
import os
from datetime import datetime


# Save Directory (Change for your own directory)
cwd = os.getcwd()

# Creating SubDirectory
exps_dir = os.path.join(cwd, 'GraphData')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

exp_name = "Lenet5"

exp_dir = os.path.join(exps_dir, exp_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# ----------------

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1,write_graph=True,write_images=True,embeddings_freq=1)  
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    callbacks.append(es_callback)

# ---------------------------------

# How to visualize Tensorboard

# 1. tensorboard --logdir EXPERIMENTS_DIR c--port PORT     <- from terminal
# 2. localhost:PORT   <- in your browser

## Training

In [ ]:
lenet.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=int(np.ceil(x_train.shape[0] / bs)),
          validation_data=valid_dataset,
          validation_steps=int(np.ceil(x_valid.shape[0] / bs)), 
          callbacks=callbacks)

## Saving Weights

In [ ]:
lenet.save_weights("C:/Users/nicol/Desktop/weights/Weights" )